使用CNN 进行训练 

In [ ]:
# print(1)

# import tensorflow as tf

1


ModuleNotFoundError: No module named 'tensorflow'

In [7]:
import pandas as pd
import numpy as np
import cv2
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
import csv
from skimage.feature import hog
import os
import xmltodict
from sklearn.pipeline import make_pipeline

def parse_xml_to_dataframe(xml_folder):
    data = []
    
    # 遍历 XML 文件夹中的每个文件
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            with open(os.path.join(xml_folder, xml_file), 'r') as file:
                xml_data = xmltodict.parse(file.read())
            
            # 提取关键信息
            annotation = xml_data['annotation']
            filename = annotation['filename']
            size = annotation['size']
            objects = annotation['object']
            
            # 提取图像尺寸
            width = int(size['width'])
            height = int(size['height'])
            
            # 如果存在多个对象，需要处理成列表
            if isinstance(objects, list):
                for obj in objects:
                    bndbox = obj['bndbox']
                    data.append({
                        'image_name': filename,
                        'width': width,
                        'height': height,
                        'xmin': int(bndbox['xmin']),
                        'ymin': int(bndbox['ymin']),
                        'xmax': int(bndbox['xmax']),
                        'ymax': int(bndbox['ymax'])
                    })
            else:
                bndbox = objects['bndbox']
                data.append({
                    'image_name': filename,
                    'width': width,
                    'height': height,
                    'xmin': int(bndbox['xmin']),
                    'ymin': int(bndbox['ymin']),
                    'xmax': int(bndbox['xmax']),
                    'ymax': int(bndbox['ymax'])
                })

    # 转换为 Pandas DataFrame
    df = pd.DataFrame(data)
    return df

# 调用函数解析 XML 数据
xml_folder = 'C:/code/vcpython/ML_design_1/task1/detection/train_location'  # 替换为实际路径
train_location_df = parse_xml_to_dataframe(xml_folder)
print(train_location_df)


def normalize_coordinates(df):
    df['xmin'] = df['xmin'] / df['width']
    df['ymin'] = df['ymin'] / df['height']
    df['xmax'] = df['xmax'] / df['width']
    df['ymax'] = df['ymax'] / df['height']
    return df

# 对边框数据归一化
train_location_df = normalize_coordinates(train_location_df)
print(train_location_df)

   image_name  width  height  xmin  ymin  xmax  ymax
0    0001.jpg   1956    1934   631   903   839  1121
1    0002.jpg   1956    1934   653   901   883  1131
2    0003.jpg   2992    2000  1368   939  1570  1137
3    0004.jpg   2992    2000  1560  1006  1757  1202
4    0005.jpg   2992    2000  1637  1016  1826  1202
..        ...    ...     ...   ...   ...   ...   ...
75   0076.jpg   2992    2000  1407   933  1582  1106
76   0077.jpg   2992    2000  1203   837  1399  1037
77   0078.jpg   2992    2000  1570  1006  1766  1199
78   0079.jpg   2992    2000  1187  1037  1355  1222
79   0080.jpg   2992    2000  1364   943  1549  1118

[80 rows x 7 columns]
   image_name  width  height      xmin      ymin      xmax      ymax
0    0001.jpg   1956    1934  0.322597  0.466908  0.428937  0.579628
1    0002.jpg   1956    1934  0.333845  0.465874  0.451431  0.584798
2    0003.jpg   2992    2000  0.457219  0.469500  0.524733  0.568500
3    0004.jpg   2992    2000  0.521390  0.503000  0.587233  0.601

使用SVM回归预测

1、导入包

In [8]:
import pandas as pd
import numpy as np
import cv2
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform
import csv
from skimage.feature import hog
import os
import xmltodict
from sklearn.pipeline import make_pipeline

2、读取数据,处理xml文件

In [9]:
def parse_xml_to_dataframe(xml_folder):
    data = []
    
    # 遍历 XML 文件夹中的每个文件
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            with open(os.path.join(xml_folder, xml_file), 'r') as file:
                xml_data = xmltodict.parse(file.read())
            
            # 提取关键信息
            annotation = xml_data['annotation']
            filename = annotation['filename']
            size = annotation['size']
            objects = annotation['object']
            
            # 提取图像尺寸
            width = int(size['width'])
            height = int(size['height'])
            
            # 如果存在多个对象，需要处理成列表
            if isinstance(objects, list):
                for obj in objects:
                    bndbox = obj['bndbox']
                    data.append({
                        'image_name': filename,
                        'width': width,
                        'height': height,
                        'xmin': int(bndbox['xmin']),
                        'ymin': int(bndbox['ymin']),
                        'xmax': int(bndbox['xmax']),
                        'ymax': int(bndbox['ymax'])
                    })
            else:
                bndbox = objects['bndbox']
                data.append({
                    'image_name': filename,
                    'width': width,
                    'height': height,
                    'xmin': int(bndbox['xmin']),
                    'ymin': int(bndbox['ymin']),
                    'xmax': int(bndbox['xmax']),
                    'ymax': int(bndbox['ymax'])
                })

    # 转换为 Pandas DataFrame
    df = pd.DataFrame(data)
    return df

# 调用函数解析 XML 数据
xml_folder = 'C:/code/vcpython/ML_design_1/task1/detection/train_location'  # 替换为实际路径
train_location_df = parse_xml_to_dataframe(xml_folder)
print(train_location_df)

   image_name  width  height  xmin  ymin  xmax  ymax
0    0001.jpg   1956    1934   631   903   839  1121
1    0002.jpg   1956    1934   653   901   883  1131
2    0003.jpg   2992    2000  1368   939  1570  1137
3    0004.jpg   2992    2000  1560  1006  1757  1202
4    0005.jpg   2992    2000  1637  1016  1826  1202
..        ...    ...     ...   ...   ...   ...   ...
75   0076.jpg   2992    2000  1407   933  1582  1106
76   0077.jpg   2992    2000  1203   837  1399  1037
77   0078.jpg   2992    2000  1570  1006  1766  1199
78   0079.jpg   2992    2000  1187  1037  1355  1222
79   0080.jpg   2992    2000  1364   943  1549  1118

[80 rows x 7 columns]


3、边框数据归一化

In [10]:
def normalize_coordinates(df):
    df['xmin'] = df['xmin'] / df['width']
    df['ymin'] = df['ymin'] / df['height']
    df['xmax'] = df['xmax'] / df['width']
    df['ymax'] = df['ymax'] / df['height']
    return df

# 对边框数据归一化
train_location_df = normalize_coordinates(train_location_df)
print(train_location_df)


   image_name  width  height      xmin      ymin      xmax      ymax
0    0001.jpg   1956    1934  0.322597  0.466908  0.428937  0.579628
1    0002.jpg   1956    1934  0.333845  0.465874  0.451431  0.584798
2    0003.jpg   2992    2000  0.457219  0.469500  0.524733  0.568500
3    0004.jpg   2992    2000  0.521390  0.503000  0.587233  0.601000
4    0005.jpg   2992    2000  0.547126  0.508000  0.610294  0.601000
..        ...    ...     ...       ...       ...       ...       ...
75   0076.jpg   2992    2000  0.470254  0.466500  0.528743  0.553000
76   0077.jpg   2992    2000  0.402072  0.418500  0.467580  0.518500
77   0078.jpg   2992    2000  0.524733  0.503000  0.590241  0.599500
78   0079.jpg   2992    2000  0.396725  0.518500  0.452874  0.611000
79   0080.jpg   2992    2000  0.455882  0.471500  0.517714  0.559000

[80 rows x 7 columns]


4、整合信息，将csv文件整合到边框中

In [ ]:
#格式化fovea_localization_train_GT.csv文件
fovea_gt_copy = pd.read_csv('C:/code/vcpython/ML_design_1/task1/detection/fovea_localization_train_GT_copy.csv')

def format_fovea_gt(df):
    for i in df['image_name']:
        
        df['image_name'][i - 1] = f'{i:04d}.jpg'
        # print(i)
    return df

fovea_gt_copy = format_fovea_gt(fovea_gt_copy)
# print(fovea_gt_copy)

fovea_gt_copy.to_csv('C:/code/vcpython/ML_design_1/task1/detection/fovea_localization_train_GT_copy.csv', index=False)

5、对黄斑区域数据进行预处理

In [17]:
from skimage.feature import hog
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import cv2
import os

# 提取图像的 HOG 特征
def extract_hog_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 转为灰度图
    img = cv2.resize(img, (256, 256))  # 调整图像大小
    features, _ = hog(img, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features

# 准备数据
def prepare_data(df, image_folder):
    features = []
    labels = []
    
    for _, row in df.iterrows():
        # 获取图像路径和黄斑区域坐标
        image_path = os.path.join(image_folder, row['image_name'])
        xmin, ymin, xmax, ymax = row['xmin'], row['ymin'], row['xmax'], row['ymax']
        
        # 提取 HOG 特征
        feature = extract_hog_features(image_path)
        
        # 将归一化坐标作为目标值
        labels.append([xmin, ymin, xmax, ymax])
        features.append(feature)
    
    return np.array(features), np.array(labels)

# 加载数据
image_folder = 'C:/code/vcpython/ML_design_1/task1/detection/train'  # 图像文件夹路径
X, y = prepare_data(train_location_df, image_folder)

# 切分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用标准化管道
svm_regressor_xmin = make_pipeline(StandardScaler(), SVR(kernel='rbf', C=1, gamma=0.01))
svm_regressor_ymin = make_pipeline(StandardScaler(), SVR(kernel='rbf', C=1, gamma=0.01))
svm_regressor_xmax = make_pipeline(StandardScaler(), SVR(kernel='rbf', C=1, gamma=0.01))
svm_regressor_ymax = make_pipeline(StandardScaler(), SVR(kernel='rbf', C=1, gamma=0.01))

# 训练模型
svm_regressor_xmin.fit(X_train, y_train[:, 0])
svm_regressor_ymin.fit(X_train, y_train[:, 1])
svm_regressor_xmax.fit(X_train, y_train[:, 2])
svm_regressor_ymax.fit(X_train, y_train[:, 3])

# 预测
y_pred_xmin = svm_regressor_xmin.predict(X_test)
y_pred_ymin = svm_regressor_ymin.predict(X_test)
y_pred_xmax = svm_regressor_xmax.predict(X_test)
y_pred_ymax = svm_regressor_ymax.predict(X_test)

# 合并预测结果
y_pred = np.stack((y_pred_xmin, y_pred_ymin, y_pred_xmax, y_pred_ymax), axis=1)

# 计算误差
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# 你也可以保存模型以便后续使用
import joblib
joblib.dump(svm_regressor_xmin, 'svm_regressor_xmin.pkl')
joblib.dump(svm_regressor_ymin, 'svm_regressor_ymin.pkl')
joblib.dump(svm_regressor_xmax, 'svm_regressor_xmax.pkl')
joblib.dump(svm_regressor_ymax, 'svm_regressor_ymax.pkl')


Mean Squared Error: 0.0026855341043829383


['svm_regressor_ymax.pkl']

6、绘制黄斑框

In [20]:
import cv2
import joblib
import numpy as np
import os

def draw_bounding_box(image_path, bbox, output_path=None):
    """
    在图像上绘制预测框
    :param image_path: 图像路径
    :param bbox: 预测框，格式为 [xmin, ymin, xmax, ymax]，归一化坐标
    :param output_path: 保存绘制结果的路径
    """
    img = cv2.imread(image_path)
    height, width = img.shape[:2]

    # 反归一化坐标
    xmin = int(bbox[0] * width)
    ymin = int(bbox[1] * height)
    xmax = int(bbox[2] * width)
    ymax = int(bbox[3] * height)

    # 绘制矩形框
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)

    # 保存结果或显示
    if output_path:
        cv2.imwrite(output_path, img)
    else:
        cv2.imshow("Prediction", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

# 加载模型
svm_regressor_xmin = joblib.load('svm_regressor_xmin.pkl')
svm_regressor_ymin = joblib.load('svm_regressor_ymin.pkl')
svm_regressor_xmax = joblib.load('svm_regressor_xmax.pkl')
svm_regressor_ymax = joblib.load('svm_regressor_ymax.pkl')

for i in range(81, 101):
    
  # 加载测试图像和提取特征
  test_image_path = 'C:/code/vcpython/ML_design_1/task1/detection/test/' + f'{i:04d}.jpg'
  test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
  test_image = cv2.resize(test_image, (128, 128))
  test_features, _ = hog(test_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)

  # 预测
  test_features = test_features.reshape(1, -1)  # 确保是 2D 输入
  predicted_xmin = svm_regressor_xmin.predict(test_features)
  predicted_ymin = svm_regressor_ymin.predict(test_features)
  predicted_xmax = svm_regressor_xmax.predict(test_features)
  predicted_ymax = svm_regressor_ymax.predict(test_features)

  predicted_bbox = [predicted_xmin[0], predicted_ymin[0], predicted_xmax[0], predicted_ymax[0]]

  # 绘制预测框
  draw_bounding_box(test_image_path, predicted_bbox)


ValueError: X has 8100 features, but StandardScaler is expecting 34596 features as input.

In [ ]:
# 加载中心凹坐标
fovea_gt = pd.read_csv('C:/code/vcpython/ML_design_1/task1/detection/fovea_localization_train_GT_copy.csv')
# print(fovea_gt.columns)

# 确保列名一致
# fovea_gt.rename(columns={'image_name': 'image_name'}, inplace=True)

# 归一化中心凹坐标
# fovea_gt['Fovea_X'] = fovea_gt['Fovea_X'] / train_location_df['width']  # 假设宽度和图像相同
# fovea_gt['Fovea_Y'] = fovea_gt['Fovea_Y'] / train_location_df['height']

# print(fovea_gt)

# 合并边框数据和中心凹坐标
train_data = pd.merge(train_location_df, fovea_gt, on='image_name')
print(train_data)


   image_name  width  height      xmin      ymin      xmax      ymax  \
0    0001.jpg   1956    1934  0.322597  0.466908  0.428937  0.579628   
1    0002.jpg   1956    1934  0.333845  0.465874  0.451431  0.584798   
2    0003.jpg   2992    2000  0.457219  0.469500  0.524733  0.568500   
3    0004.jpg   2992    2000  0.521390  0.503000  0.587233  0.601000   
4    0005.jpg   2992    2000  0.547126  0.508000  0.610294  0.601000   
..        ...    ...     ...       ...       ...       ...       ...   
75   0076.jpg   2992    2000  0.470254  0.466500  0.528743  0.553000   
76   0077.jpg   2992    2000  0.402072  0.418500  0.467580  0.518500   
77   0078.jpg   2992    2000  0.524733  0.503000  0.590241  0.599500   
78   0079.jpg   2992    2000  0.396725  0.518500  0.452874  0.611000   
79   0080.jpg   2992    2000  0.455882  0.471500  0.517714  0.559000   

        Fovea_X      Fovea_Y  
0    734.031331  1016.817066  
1    769.024859  1018.059762  
2   1470.287475  1037.601074  
3   1658.31

5、存储清洗过后的数据

In [ ]:
train_data.to_csv('cleaned_train_data.csv', index=False)


6、使用SVM进行训练

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_squared_error

# 读取数据
data = pd.read_csv('cleaned_train_data.csv', header=None)
data.columns = ['image_name', 'width', 'height', 'x1', 'y1', 'x2', 'y2', 'fovea_x', 'fovea_y']

# 添加一个标签列，假设中心凹的框是由x1, y1, x2, y2定义的
data['label'] = 1  # 假设所有数据都是中心凹的框

# 特征和目标变量
X = data[['width', 'height', 'x1', 'y1', 'x2', 'y2']]
y_class = data['label']
y_reg = data[['fovea_x', 'fovea_y']]

# 分割数据集
X_train, X_test, y_class_train, y_class_test, y_reg_train, y_reg_test = train_test_split(X, y_class, y_reg, test_size=0.2, random_state=42)

# 训练SVM分类模型
svc = SVC(kernel='rbf')
svc.fit(X_train, y_class_train)



Classification Accuracy: 0.46875
Mean Squared Error for fovea_x: 23935.13774945755
Mean Squared Error for fovea_y: 1909.9304253350995
Predicted fovea coordinates: (1469.5253168020552, 1047.6424107564362)


C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
